In [1]:
import pandas as pd

df= pd.read_csv("C://Users//Manisha Chandanshiv//OneDrive//ドキュメント//Excelr assignment Questions//Data Science//Naive Bayes and Text Mining//Naive Bayes and Text Mining//blogs_categories.csv")

df.head()

,Unnamed: 0,Data,Labels
0,0,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49...,alt.atheism
1,1,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
2,2,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
3,3,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
4,4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism


In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '',text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['cleaned'] = df['Data'].apply(preprocess)


[nltk_data] Downloading package stopwords to C:\Users\Manisha
[nltk_data]     Chandanshiv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Manisha
[nltk_data]     Chandanshiv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X=tfidf.fit_transform(df['cleaned']).toarray()
y= df['Labels']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)


In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] > 0.05:
        return 'positive'
    elif score['compound'] < -0.05:
        return 'negative'
    else:
        return 'neutral'

df['Sentiment'] = df['Data'].apply(get_sentiment)


[nltk_data] Downloading package vader_lexicon to C:\Users\Manisha
[nltk_data]     Chandanshiv\AppData\Roaming\nltk_data...


In [15]:
sentiment_dist = df.groupby('Labels')['Sentiment'].value_counts(normalize=True).unstack().fillna(0)
print(sentiment_dist)


Sentiment                 negative  neutral  positive
Labels                                               
alt.atheism               0.379000    0.008  0.613000
comp.graphics             0.123000    0.047  0.830000
comp.os.ms-windows.misc   0.204000    0.045  0.751000
comp.sys.ibm.pc.hardware  0.218000    0.019  0.763000
comp.sys.mac.hardware     0.243000    0.047  0.710000
comp.windows.x            0.226000    0.040  0.734000
misc.forsale              0.126000    0.070  0.804000
rec.autos                 0.304000    0.028  0.668000
rec.motorcycles           0.311000    0.020  0.669000
rec.sport.baseball        0.228000    0.039  0.733000
rec.sport.hockey          0.259000    0.016  0.725000
sci.crypt                 0.300000    0.007  0.693000
sci.electronics           0.178000    0.036  0.786000
sci.med                   0.346000    0.022  0.632000
sci.space                 0.271000    0.021  0.708000
soc.religion.christian    0.260782    0.000  0.739218
talk.politics.guns        0.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
